<a href="https://colab.research.google.com/github/Ankan1998/Sarcasm_detector/blob/master/Sarcasm_Detector_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os 
import zipfile
import pandas as pd
import numpy as np

In [2]:
!pip install transformers

     |████████████████████████████████| 890kB 5.4MB/s 
     |████████████████████████████████| 3.0MB 28.9MB/s 
     |████████████████████████████████| 1.1MB 58.8MB/s 
     |████████████████████████████████| 890kB 54.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cebcaccaa1919bd31efe3cc176aad821ba71a87cd2a035f7e5d3dd0ae0c068f0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

# **Sarcasm Detector using NLP**

In [4]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [5]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-cp36-none-any.whl size=73275 sha256=16d0af69a9a17e242234da46721358b553ce9f7da61371cb2f36c14e61263981
  Stored in directory: /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c
Successfully built kaggle
  Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8


In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

#### Importing kaggle file

In [7]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection -p "/tmp"

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 240MB/s]


#### Extract File

In [9]:
local_zip='/tmp/news-headlines-dataset-for-sarcasm-detection.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall("/tmp")
zip_ref.close()

## **Data Preparation**

In [47]:
import json

def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('/tmp/Sarcasm_Headlines_Dataset.json'))

training_size = 25000

sentences = []
labels = []
urls = []
for item in data:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

In [40]:
len(training_sentences)

25000

In [41]:
len(validation_sentences)

1709

## **Starting preprocessing on sentences**

In [48]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [49]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [50]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

##**Applying distilBERT quantized simple, lighter BERT**

In [51]:
# We classify two labels in this example. In case of multiclass classification, adjust num_labels value
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                              num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_79', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [52]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
trainer=model.fit(train_dataset.shuffle(42).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(42).batch(16))

Epoch 1/2
1563/1563 [==============================] - 199s 127ms/step - loss: 0.2512 - accuracy: 0.8930 - val_loss: 0.2598 - val_accuracy: 0.9023
Epoch 2/2
1563/1563 [==============================] - 197s 126ms/step - loss: 0.0886 - accuracy: 0.9675 - val_loss: 0.2565 - val_accuracy: 0.9040
